In [250]:
from collections import Counter
import pandas as pd
import numpy as np
import time
import datetime

In [289]:
df = pd.read_csv('Event_log.txt', sep='\t', encoding='cp1251')

In [290]:
df.head()

,CaseID,Event end,Activity,Activity Category,Activity Class,Activity type,User,Department,Amount,Purchase type,Supplier,Supplier risk level,Supplier Size,Supplier_type,INVOICE,Cleared invoice,PO TYPE,Material,User_type
0,2519 - 236,2017-04-25 08:52:27.000,Авансовый платеж,Авансовый платеж,Авансовый платеж,Авансовый платеж,NaN,Uncategorized,144.775,Услуги,Sellers of Cycles,Low,Unknown,Финансовые учреждения,NaN,NaN,Заказ на оформление закупки,Credit for Black Touring End Caps,NaN
1,2519 - 236,2017-04-24 10:32:53.000,Требование авансового платежа,Авансовый платеж,Авансовый платеж,Авансовый платеж,NaN,Uncategorized,144.775,Услуги,Sellers of Cycles,Low,Unknown,Финансовые учреждения,NaN,NaN,Заказ на оформление закупки,Credit for Black Touring End Caps,NaN
2,2519 - 236,2017-04-25 08:59:34.000,Перерасчет авансового платежа,Перерасчет авансового платежа,Платеж (выравнивание),Перерасчет авансового платежа,NaN,Uncategorized,144.775,Услуги,Sellers of Cycles,Low,Unknown,Финансовые учреждения,NaN,NaN,Заказ на оформление закупки,Credit for Black Touring End Caps,NaN
3,2519 - 236,2017-04-25 08:58:14.000,Поступление счета,Поступление счета,Счет,Поступление счета,User_111,Accounting Group,144.775,Услуги,Sellers of Cycles,Low,Unknown,Финансовые учреждения,NaN,NaN,Заказ на оформление закупки,Credit for Black Touring End Caps,A
4,2519 - 236,2017-04-25 08:52:27.000,Перерасчет по требованию авансового платежа,Перерасчет авансового платежа,Платеж (выравнивание),Перерасчет авансового платежа,User_111,Accounting Group,144.775,Услуги,Sellers of Cycles,Low,Unknown,Финансовые учреждения,NaN,NaN,Заказ на оформление закупки,Credit for Black Touring End Caps,A


In [291]:
df['Event end'] = pd.to_datetime(df['Event end'])

In [269]:
print("Количество кэйсов:",len(df['CaseID'].unique()))

Количество кэйсов: 8025


Уникальные кейсы.

In [270]:
cases = df['CaseID'].unique()
cases

array(['2519 - 236', '2137 - 1045', '2293 - 429', ..., '7666 - 1003',
       '7471 - 644', '6366 - 999'], dtype=object)

In [271]:
classes = df['Activity Class'].unique()
classes

array(['Авансовый платеж', 'Платеж (выравнивание)', 'Счет',
       'Заказ на поставку', 'Заявка', 'Поступление материала',
       'Учет услуг'], dtype=object)

## KPI

На вход подается объект - dataframe.

### Общие.

1) О-1 Самые популярные категории событий. +

2) О-2 Процент завершившихся цепочек. +

3) О-3 Сложные цепочки. -+ (время работы)

4) О-4 Общее количество кейсов. +

5) О-5 Общее количество уникальных пользователей. +

6) О-6 Уникальные цепочки. -+ (проблема сортировки)

7) О-7 Нестандартные цепочки. -+ (зависит от уникальных маршрутов)

8) О-8 Среднее количество цепочек, созданных за день. +

9) О-9 Среднее количество времени на цепочку проведенное в рамках одного класса. +- (медленно работает + считает совпадающие по времени события)

10) О-10 Среднее количество циклов на цепочку. -+ (медленно работает + зависит от уникальных маршрутов)

#### 1. О-1 Самые популярные категории событий.

In [209]:
def O_1(df):
    #Вычисляет популярные категории событий относительно количества выполненных событий и пользователей.
    
    #Вход:  dataframe - eventlog.
    #Выход: Кортеж (act_am_dict, users_am_dict) 
    #        act_am_dict - словарь, в котором ключ - название категории событий('Activity Category'),
    #        а значение - отношение количества событий, входящих в эту категорию, и количества всех событий.
    #        users_am_dict - словарь, в котором ключ - название категории событий('Activity Category'),
    #        а значение - количество уникальных пользователей, которые участвовали в этой категории событий.
    
    act_am_dict = df.groupby('Activity Category')['Activity'].agg(['count']).div(df.shape[0]).to_dict()['count']
    users_am_dict = df.groupby('Activity Category').agg({"User": lambda x: x.nunique()}).to_dict()['User']
    sorted_act_am_dict = sorted(act_am_dict.items(), key=lambda kv: kv[1], reverse=True)
    sorted_users_am_dict = sorted(users_am_dict.items(), key=lambda kv: kv[1], reverse=True)
    
    return sorted_act_am_dict[:5], sorted_users_am_dict[:5]

In [210]:
#time check
%time O_1(df)

CPU times: user 47.6 ms, sys: 25.5 ms, total: 73.2 ms
Wall time: 89.2 ms


([('Поступление счета', 0.14415557922890365),
  ('Платеж (выравнивание)', 0.12893537931503435),
  ('Счет изменен', 0.11527913143996918),
  ('Статус счета изменен', 0.09923162356354405),
  ('Заказ на поставку изменен', 0.0967157007185113)],
 [('Заказ на поставку создан', 87),
  ('Заказ на поставку изменен', 55),
  ('Поступление материала-Получение', 36),
  ('Поступление материала-Возврат', 26),
  ('Заказ на поставку согласован', 19)])

#### 2. О-2 Процент завершившихся цепочек.

In [211]:
def O_2(df):
    #Считает процент начатых и незаконченных цепочек от общего числа цепочек.
    
    #Вход:  dataframe - eventlog.
    #Выход: отношение кол-ва завершенных цепочек(т.е. таких цепочек, которые закончились Катагорией События
    #       ('Activity Category') == 'Платеж (выравнивание)') и кол-ва всех цепочек.
    
    started_cases = set(df.iloc[np.where(df['Activity Category'] == 'Заказ на поставку создан')]['CaseID'])
    finished_cases = set(df.iloc[np.where(df['Activity Category'] == 'Платеж (выравнивание)')]['CaseID'])
    
    return len(started_cases & finished_cases) / len(df['CaseID'].unique())

In [212]:
#time check
%time O_2(df)

CPU times: user 46.7 ms, sys: 24 ms, total: 70.8 ms
Wall time: 88.3 ms


0.9112772585669782

#### 3. О-3 Сложные цепочки.

In [213]:
# -долго работает

def convert_to_adj_list(activ):
    #Переводит из последовательности событий в список смежности.(более удобное представление графа).
    
    #Вход:  activ - list, массив строк - последовательность событий одной цепочки.
    #Выход: res - list, массив смежности графа.
    decode = list(set(activ))
    dec_dict = {}
    
    for i in range(len(decode)):
        dec_dict[decode[i]] = i
    
    res = [[] for i in range(len(decode))]
    
    for i in range(len(activ) - 1):
        res[dec_dict[activ[i]]].append(dec_dict[activ[i + 1]])
    
    return res

def O_3(df):
    # оценить сложность - просчитать 
    
    #Считает количество сложных цепочек. Сложная цепочка - такая цепочка, которая отклоняется от среднего
    #больше, чем на одно стандартное отклонение, по количеству кейсов и количеству циклов. 
    
    #Вход:  dataframe-eventlog.
    #Выход: количество сложных цепочек.
    #       number of atypical chains. Atypical chain - chain, that has more than one standart
    #       deviation from mean value of activity and loop amount. 
    num_of_act_in_case = df.groupby('CaseID').agg({'Activity': 'count'})['Activity'].to_dict()
    
    num_of_loop_in_case = {}

    cases = df['CaseID'].unique()
    begin = time.time()
    for case in cases:
        chain = df.iloc[np.where(df['CaseID'] == case)]
        chain = chain.sort_values(by='Event end')
        activ = chain['Activity'].tolist()
    
        l_am = 0
    
        if len(activ) != len(set(activ)):
 
            conv_activ = convert_to_adj_list(activ)
            visited = [0 for i in range(len(conv_activ))]
        
            def dfs(vert, l_am):
                #Обход в глубину.
                visited[vert] = 1
                for v in conv_activ[vert]:
                    if not visited[v]:
                        l_am = dfs(v, l_am)
                    else:
                        l_am += 1
                return l_am

            for i in range(len(conv_activ)):
                if not visited[i]:
                    l_am += dfs(i, l_am)

        num_of_loop_in_case[case] = l_am
        
    loop_in_case = np.array(list(num_of_loop_in_case.values()))
    act_in_case = np.array(list(num_of_act_in_case.values()))
    
    av_loop = np.mean(loop_in_case)
    std_loop = np.std(loop_in_case) 
    ind_loop = np.where(loop_in_case > av_loop + 2 * std_loop)

    av_act = np.mean(act_in_case)
    std_act = np.std(act_in_case)
    ind_act = np.where(act_in_case > av_act + 2 * std_act)
    
    return len(set(ind_act[0]) & set(ind_loop[0]))

In [214]:
%time O_3(df)

CPU times: user 58.2 s, sys: 614 ms, total: 58.8 s
Wall time: 59.6 s


6

#### 4. О-4 Общее количество цепочек.

In [215]:
def O_4(df):
    #Вычисляет общее количество цепочек.
    
    #Вход:  dataframe - eventlog.
    #Выход: общее количество цепочек.
    
     return len(df['CaseID'].unique())

In [216]:
#time check
%time O_4(df)

CPU times: user 6.66 ms, sys: 500 µs, total: 7.16 ms
Wall time: 7.11 ms


8025

#### 5. О-5 Общее количество уникальных пользователей.

In [217]:
def O_5(df):
    #Вход:  dataframe - eventlog.
    #Выход: количество уникальных пользователей.
    
    return len(df['User'].dropna().unique())

In [218]:
#time check
%time O_5(df)

CPU times: user 16.3 ms, sys: 3.98 ms, total: 20.2 ms
Wall time: 20 ms


123

#### 6. О-6 Уникальные цепочки.

In [294]:
#считает словарь ключ - уникальная цепочка, значение - количество повторений, т.е. сразу считает "гистрограмму"
#популярности цепочек
#замечание! медленно работает - ускорить

# eq_chains - события одного маршрута
# num_cases - CaseID цепочек, которые входят в маршрут
# d - словарь, в котором ключ - номер маршрута, значение - количество цеопчек этого маршрута 
# k - сколько всего маршрутов на текущий момент
# i - число для проверки того, прошли ли мы все значения словря за итерацию

def O_6(df, sort=False):
    #Считает количество уникальных цепочек.
    
    #Вход:  dataframe - eventlog, 
    #       sort - булевая переменная, указывает, отсортирован ли по времени входные данные.
    #Выход: количество уникальных цепочек.
    
    cases = df['CaseID'].unique()
    chain = df.iloc[np.where(df['CaseID'] == cases[0])]
    if not sort:
        chain = chain.sort_values(by=['Event end'])
    activ = chain['Activity'].tolist()

    eq_chains = []
    eq_chains.append(activ)

    num_cases = [[] for i in range(len(cases))]
    num_cases[0].append(cases[0])

    d = {}
    d[0] = 1
    k = 1

    for case in cases[1:]:
        chain = df.iloc[np.where(df['CaseID'] == case)]
        if not sort:
            chain = chain.sort_values(by=['Event end'])
        activ = chain['Activity'].tolist()
        i = 0
    
        for key in d.keys():
            if eq_chains[key] == activ:
                d[key] += 1
                num_cases[key].append(case)
                break
            if i == k:
                break
            i += 1
        if i == k:
            d[k] = 1
            eq_chains.append(activ)
            num_cases[k].append(case)
            k += 1
    return d, eq_chains, num_cases

Рассмотрим более детально проблему в двух подходах сортировки. 

Первый - сначала сортируем весь датафрейм, затем работаем с цепочками.

Второй - сначала фиксируем цепочку и сортируем ее.

In [295]:
df_time_sort = df.sort_values(by=['Event end'])

#%time O_6(df)
first_dict, first_chains, first_cases = O_6(df_time_sort, True)

In [296]:
len(first_dict)

3143

In [297]:
#%time O_6(df)
second_dict, second_chains, second_cases = O_6(df)

In [298]:
len(second_dict)

2986

Введем третий подход - сортировка по двум столбцам, сначала по *'Event end'*, затем по *'Activity'*.

Подход заведомо неверный с точки зрения логики процесса, однако это в некотором роде "стандартизация" сортировки при одинаковой временной отметке.

Посмотрим, какой результат будет при данном подходе.

In [301]:
df_new_sort = df.sort_values(by=['Event end', 'Activity'])
third_dict, third_chains, third_cases = O_6(df_new_sort, True)

In [302]:
len(third_dict)

1660

In [303]:
first_cases[3]

['118 - 282', '1765 - 461', '5538 - 317']

In [304]:
for i in range(len(second_cases)):
    if '118 - 282' in second_cases[i]:
        print(i)

79


In [306]:
first_chains[3]

['Создание Заявки',
 'Заявка изменена: количество',
 'Заявка изменена: дата доставки',
 'Заявка согласована',
 'Заявка изменена: закупочная организация',
 'Заказ на поставку создан',
 'Заказ на поставку: изменен статус выпуска: ДанныеСогл, измен не возможны',
 'Заказ на поставку согласован 2',
 'Поступление материала-Получение-Завершающая поставка',
 'Поступление счета',
 'Счет изменен: условия оплаты',
 'Счет изменен: дата',
 'Счет заведен',
 'Платеж (выравнивание)']

In [307]:
second_chains[79]

['Создание Заявки',
 'Заявка изменена: дата доставки',
 'Заявка изменена: количество',
 'Заявка согласована',
 'Заявка изменена: закупочная организация',
 'Заказ на поставку создан',
 'Заказ на поставку: изменен статус выпуска: ДанныеСогл, измен не возможны',
 'Заказ на поставку согласован 2',
 'Поступление материала-Получение-Завершающая поставка',
 'Счет изменен: условия оплаты',
 'Счет заведен',
 'Счет изменен: дата',
 'Поступление счета',
 'Платеж (выравнивание)']

In [308]:
second_cases[79] 

['6208 - 508',
 '4116 - 457',
 '5344 - 521',
 '3752 - 950',
 '996 - 1044',
 '118 - 282',
 '2490 - 171']

In [310]:
df.iloc[np.where(df['CaseID'] == '6208 - 508')].sort_values(by='Event end')


,CaseID,Event end,Activity,Activity Category,Activity Class,Activity type,User,Department,Amount,Purchase type,Supplier,Supplier risk level,Supplier Size,Supplier_type,INVOICE,Cleared invoice,PO TYPE,Material,User_type
70025,6208 - 508,2017-10-20 00:00:00,Создание Заявки,Создание Заявки,Заявка,Создание Заявки,User_127,Uncategorized,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",NaN
68580,6208 - 508,2017-10-23 09:50:37,Заявка изменена: дата доставки,Заявка изменена,Заявка,Заявка изменена,User_39,Uncategorized,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",A
68581,6208 - 508,2017-10-23 09:50:37,Заявка изменена: количество,Заявка изменена,Заявка,Заявка изменена,User_39,Uncategorized,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",A
68587,6208 - 508,2017-10-23 11:43:36,Заявка согласована,Заявка согласована,Заявка,Заявка согласована,User_39,Uncategorized,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",A
68582,6208 - 508,2017-10-23 14:01:14,Заявка изменена: закупочная организация,Заявка изменена,Заявка,Заявка изменена,User_62,Managed Item Procurement Department,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",A
70024,6208 - 508,2017-10-23 14:02:39,Заказ на поставку создан,Заказ на поставку создан,Заказ на поставку,Заказ на поставку создан,User_62,Managed Item Procurement Department,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",A
68584,6208 - 508,2017-11-17 13:11:23,Заказ на поставку: изменен статус выпуска: Дан...,Заказ на поставку изменен,Заказ на поставку,Заказ на поставку: изменен статус выпуска: воз...,User_62,Managed Item Procurement Department,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",A
68585,6208 - 508,2017-11-17 13:11:23,Заказ на поставку согласован 2,Заказ на поставку согласован,Заказ на поставку,Заказ на поставку согласован 2,User_62,Managed Item Procurement Department,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",A
863,6208 - 508,2017-11-20 13:48:27,Поступление материала-Получение-Завершающая по...,Поступление материала-Получение,Поступление материала,Поступление материала-Получение,User_133,Warehouse of materials,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,"Green Classic Vest, M",A
68579,6208 - 508,2017-11-24 09:01:14,Счет изменен: условия оплаты,Счет изменен,Счет,Счет изменен,User_44,Department of accounting of inventories and no...,190501.272,Товары,First Cycle Store,Low,Large,Поставщики СНГ,17413.0,X,Заказ на оформление закупки,"Green Classic Vest, M",A


Заметим, что проблема двух подходов заключается в том, что они по-разному обрабатывают случаи одинакового времени в точности до секунд, поэтому и получаются разные последовательности в цепочках, а затем и разные показатели по всем уникальным маршрутам. 

#### 7. О-7 Нестандартные цепочки по времени.

In [219]:
def O_7(df):
    # в рамках одного маршрута 
    
    # Вычисляет нестандартные цепочки по времени.
    # Нестандартная цепочка - цепочка, которая отклоняется по времени от среднего
    # более чем на два стандартных отклонения.
    
    # Вход:  dataframe-eventlog.
    # Выход: кортеж(tuple). 
    #        Первое значение - количество нестандартных по времени цепочек.
    #        Второе значение - процентное соотношение этих цепочек ко всем цепочкам.
    
    cases = df['CaseID'].unique()
    all_time = df.groupby(
        ['CaseID'])['Event end'].apply(
        lambda a: (a.max() - a.min()).total_seconds()
    )
    av = np.mean(all_time)
    std = np.std(all_time)
    ind = np.where(all_time > av + 2 * std)
    
    return len(ind[0]), len(ind[0]) / len(cases)

In [220]:
%time O_7(df)

CPU times: user 2.77 s, sys: 73.8 ms, total: 2.84 s
Wall time: 2.81 s


(373, 0.0464797507788162)

#### 8. О-8 Среднее количество цепочек, созданных за день.

In [221]:
def O_8(df):
    #Вычисляет среднее количество цепочек, созданных за день.(т.е. таких, у которых Событие('Activity') == 
    #'Заказ на поставку создан')
    
    #Вход:  dataframe - eventlog.
    #Выход: среднее количество цепочек, созданных за день.
    
    df_n = pd.DataFrame(df, columns=['Event end', 'CaseID', 'Activity'])
    #df_n['Event end'] = df_n['Event end'].apply(lambda x: x.replace(hour=0, minute=0, second=0))
    df_n['Event end'] = df_n['Event end'].astype(str).apply(lambda x: x[:10])
    am = sum(Counter(df_n.iloc[np.where(df_n['Activity'] == 'Заказ на поставку создан')]['Event end']).values())
    days = len(df_n['Event end'].unique())
    
    return am / days


In [222]:
#time check
%time O_8(df)

CPU times: user 261 ms, sys: 19.8 ms, total: 281 ms
Wall time: 285 ms


28.57857142857143

#### 9. О-9 Среднее количество времени проведенное в рамках одного класса.

In [254]:
def O_9(df):
    df.sort_values(by='Event end', inplace=True)
    all_time = {cat : 0 for cat in df['Activity Category'].unique()}
    cases = df['CaseID'].unique()
    
    for case in cases:
        chain = df.iloc[np.where(df['CaseID'] == case)]
        cat = chain['Activity Category'].tolist()
        time = chain['Event end'].tolist()
        
        for i in range(len(cat) - 2):
            all_time[cat[i]] += (time[i + 1] - time[i]).total_seconds()
    return all_time

In [255]:
%time O_9(df)

CPU times: user 1min, sys: 763 ms, total: 1min
Wall time: 1min 7s


{'Авансовый платеж': 185944531.0,
 'Авансовый платеж-Возврат': 534176.0,
 'Заказ на поставку восстановлен': 762716.0,
 'Заказ на поставку изменен': 1252116253.0,
 'Заказ на поставку отклонен': 9986001.0,
 'Заказ на поставку согласован': 872875591.0,
 'Заказ на поставку создан': 3488345320.0,
 'Заказ на поставку удален': 14580427.0,
 'Заявка восстановлена': 0.0,
 'Заявка изменена': 106069604.0,
 'Заявка отклонена': 925535.0,
 'Заявка согласована': 88439573.0,
 'Заявка удалена': 5189867.0,
 'Перерасчет авансового платежа': 68577335.0,
 'Платеж (выравнивание)': 9276520790.0,
 'Поступление материала-Возврат': 29520862.0,
 'Поступление материала-Получение': 1150662437.0,
 'Поступление счета': 541335375.0,
 'Поступление счета-Возврат': 8974283.0,
 'Создание Заявки': 377727036.0,
 'Статус счета изменен': 195171839.0,
 'Счет блокирован': 26340063.0,
 'Счет изменен': 235206989.0,
 'Учет услуг': 21688731.0}

Заметим, что у категории 'Заявка восстановлена' среднее время 0. Рассмотрим цепочки с данной категорией.

In [263]:
cases = df[df['Activity Category'] == 'Заявка восстановлена']['CaseID']
len(cases)

2

In [264]:
df[df['CaseID'] == cases.values[0]].sort_values(by='Event end')

,CaseID,Event end,Activity,Activity Category,Activity Class,Activity type,User,Department,Amount,Purchase type,Supplier,Supplier risk level,Supplier Size,Supplier_type,INVOICE,Cleared invoice,PO TYPE,Material,User_type
76588,7165 - 591,2017-04-04 00:00:00,Создание Заявки,Создание Заявки,Заявка,Создание Заявки,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76571,7165 - 591,2017-04-04 14:35:05,Заявка согласована,Заявка согласована,Заявка,Заявка согласована,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76563,7165 - 591,2017-04-04 14:39:48,Заказ на поставку создан,Заказ на поставку создан,Заказ на поставку,Заказ на поставку создан,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76578,7165 - 591,2017-04-04 14:41:42,Заказ на поставку: изменен статус выпуска: Дан...,Заказ на поставку изменен,Заказ на поставку,Заказ на поставку: изменен статус выпуска: воз...,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76585,7165 - 591,2017-04-04 14:41:42,Заказ на поставку согласован 2,Заказ на поставку согласован,Заказ на поставку,Заказ на поставку согласован 2,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76561,7165 - 591,2017-04-28 15:24:16,Заказ на поставку: изменен статус выпуска: Дан...,Заказ на поставку изменен,Заказ на поставку,Заказ на поставку: изменен статус выпуска: изм...,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76570,7165 - 591,2017-04-28 15:24:16,Заказ на поставку: согласование отклонено,Заказ на поставку отклонен,Заказ на поставку,Заказ на поставку: согласование отклонено,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76586,7165 - 591,2017-04-28 15:25:35,Заказ на поставку: согласование отклонено,Заказ на поставку отклонен,Заказ на поставку,Заказ на поставку: согласование отклонено,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76577,7165 - 591,2017-04-28 15:25:35,Заказ на поставку удален,Заказ на поставку удален,Заказ на поставку,Заказ на поставку удален,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A
76587,7165 - 591,2017-04-28 15:25:35,Заказ на поставку бессрочно заблокирован,Заказ на поставку отклонен,Заказ на поставку,Заказ на поставку блокирован,User_43,Uncategorized,NaN,Услуги,Recreation Supplies,Low,Large,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Repair of,A


In [265]:
df[df['CaseID'] == cases.values[1]].sort_values(by='Event end')

,CaseID,Event end,Activity,Activity Category,Activity Class,Activity type,User,Department,Amount,Purchase type,Supplier,Supplier risk level,Supplier Size,Supplier_type,INVOICE,Cleared invoice,PO TYPE,Material,User_type
67527,5391 - 103,2017-08-31 00:00:00,Создание Заявки,Создание Заявки,Заявка,Создание Заявки,User_127,Uncategorized,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,NaN
60588,5391 - 103,2017-09-06 21:11:06,Заявка изменена: дата доставки,Заявка изменена,Заявка,Заявка изменена,User_39,Uncategorized,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A
60582,5391 - 103,2017-09-06 21:11:22,Заявка согласована,Заявка согласована,Заявка,Заявка согласована,User_39,Uncategorized,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A
60592,5391 - 103,2017-09-07 08:13:53,Заявка изменена: закупочная организация,Заявка изменена,Заявка,Заявка изменена,User_35,Managed Item Procurement Department,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A
60590,5391 - 103,2017-09-13 09:09:00,Заявка удалена,Заявка удалена,Заявка,Заявка удалена,User_39,Uncategorized,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A
60594,5391 - 103,2017-09-13 13:58:43,Заявка восстановлена,Заявка восстановлена,Заявка,Заявка восстановлена,User_39,Uncategorized,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A
60593,5391 - 103,2017-09-13 13:58:43,Заявка изменена: дата доставки,Заявка изменена,Заявка,Заявка изменена,User_39,Uncategorized,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A
60589,5391 - 103,2017-09-14 10:43:13,Заказ на поставку создан,Заказ на поставку создан,Заказ на поставку,Заказ на поставку создан,User_35,Managed Item Procurement Department,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A
60583,5391 - 103,2017-09-14 10:43:37,Заказ на поставку согласован 2,Заказ на поставку согласован,Заказ на поставку,Заказ на поставку согласован 2,User_35,Managed Item Procurement Department,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A
60584,5391 - 103,2017-09-14 10:43:37,Заказ на поставку: изменен статус выпуска: Дан...,Заказ на поставку изменен,Заказ на поставку,Заказ на поставку: изменен статус выпуска: воз...,User_35,Managed Item Procurement Department,108.694,Товары,Norstan Bike Hut,Low,Small,Поставщики СНГ,NaN,NaN,Заказ на оформление закупки,Pink Lock Washer 10,A


Проблема заключается в том, что временные отметки данной категории совпадают с последующей отметкой, что вызывает соответствующий эффект. Также заметим, что это лишь видимый случай. В других категориях учитываются такие же совпадения по времени.

#### 10. О-10 Среднее количество циклов на цепочку.

In [223]:
def O_10(df):
    #Смотреть по маршрутам. 
    
    
    #Вычисляет среднее количество циклов на одну цепочку.
    
    #Вход:  dataframe - eventlog.
    #Выход: среднее количество циклов на цепочку.
    cases = df['CaseID'].unique()
    
    loops = []
    
    for case in cases:
        chain = df.iloc[np.where(df['CaseID'] == case)]
        chain = chain.sort_values(by='Event end')
        activ = chain['Activity'].tolist()
        
        events = set()
        loop_am = 0
        
        for act in activ:
            if act in events:
                loop_am += 1
                events.add(act)
            else:
                events.add(act)
                
        loops.append(loop_am)
        
    return np.array(loops).mean()

In [224]:
%time O_10(df)

CPU times: user 53.6 s, sys: 260 ms, total: 53.8 s
Wall time: 53.9 s


2.8697819314641744